### Configurig Models and Vars

In [ ]:
import sys
import os

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output

client, model_name, api_provider = setup_llm_client(model_name="gemini-2.5-pro")

# Load the application code from Day 3 to provide context for test generation
# TODO: potentially change this path depending on project structure
app_code = load_artifact("app/main.py")
if not app_code:
    print("Warning: Could not load app/main.py. Lab may not function correctly.")

### Generating Tests with Fixture
**Task:** Generate `pytest` tests for the ideal or "happy path" scenarios of your CRUD endpoints.

In [ ]:
# TODO: CHANGE THIS PROMPT FOR OUR FASTAPI
# TODO: Write a prompt to generate the pytest fixture for an isolated test database.
db_fixture_prompt = f"""
You are a senior QA Engineer. Generate a pytest configuration file (conftest.py) for isolated database testing in a FastAPI application.

FastAPI Application Code:
{app_code}

Requirements:
1. Create a conftest.py file that contains pytest fixtures for database isolation
2. Use an in-memory SQLite database (sqlite:///:memory:) with SQLAlchemy
3. Import necessary modules: pytest, TestClient from fastapi.testclient, create_engine, sessionmaker, StaticPool
4. Import from main.py: app, Base, get_db
5. Create a database engine with StaticPool for in-memory SQLite
6. Create a TestingSessionLocal sessionmaker bound to the test engine
7. Create an override_get_db() function that yields a test database session and closes it
8. Override the app's get_db dependency: app.dependency_overrides[get_db] = override_get_db
9. Create a pytest fixture named "client" with scope="function" that:
   - Creates all database tables using Base.metadata.create_all(bind=engine)
   - Yields a TestClient(app) instance
   - Drops all tables using Base.metadata.drop_all(bind=engine) after the test

Output only valid Python code for conftest.py with proper imports and fixtures. This file will be used by all test files.
"""

print("--- Generating Pytest DB Fixture ---")
if app_code:
    generated_db_fixture = get_completion(db_fixture_prompt, client, model_name, api_provider)
    cleaned_fixture = clean_llm_output(generated_db_fixture, language='python')
    print(cleaned_fixture)
    save_artifact(cleaned_fixture, "tests/conftest.py")
else:
    print("Skipping fixture generation because app context is missing.")

# TODO: Write a prompt to refactor the happy path tests to use the new fixture.
refactor_tests_prompt = f"""
You are a senior QA Engineer. Create happy path tests to use pytest fixtures from conftest.py.

FastAPI Application Code:
{app_code}

Requirements:
1. Import only pytest and TestClient (do NOT import database setup code)
2. Import the TestClient type hint for function parameters
3. Use the "client" fixture from conftest.py as a parameter in test functions
4. Remove all database setup code (engine, sessionmaker, override_get_db, etc.) as this is now in conftest.py
5. Keep the same test logic and assertions as the original happy path tests
6. Ensure tests use the client fixture parameter instead of a global client

Output only valid Python pytest functions that use the client fixture from conftest.py. Use descriptive function names: test_create_user and test_get_users.
"""

print("\n--- Generating Refactored Tests ---")
if app_code:
    refactored_tests = get_completion(refactor_tests_prompt, client, model_name, api_provider)
    cleaned_refactored_tests = clean_llm_output(refactored_tests, language='python')
    print(cleaned_refactored_tests)
    save_artifact(cleaned_refactored_tests, "tests/test_main_with_fixture.py")
else:
    print("Skipping test refactoring because app context is missing.")

### Generating Edge Cases

In [ ]:
# TODO: CHANGE THIS PROMPT and ADD GENERATED EDGE CASES TO TESTING FILE
edge_case_tests_prompt = f"""
You are a senior QA Engineer. Generate pytest test functions for edge cases and error scenarios in a FastAPI application.

FastAPI Application Code:
{app_code}

Requirements:
1. Use FastAPI's TestClient from fastapi.testclient import TestClient
2. Import the FastAPI app instance (same setup as happy path tests)
3. Write a test function for POST /users/ that:
   - First creates a user with a valid email
   - Then attempts to create another user with the same email
   - Asserts status code is 400 (Bad Request)
   - Verifies the error message indicates duplicate email
4. Write a test function for GET /users/{{user_id}} that:
   - Requests a user ID that does not exist (e.g., 99999 or a high number)
   - Asserts status code is 404 (Not Found)
   - Verifies an appropriate error message

Output only valid Python pytest functions with proper imports. Use descriptive function names like test_create_user_duplicate_email and test_get_user_not_found. Ensure tests use the same database setup pattern as happy path tests.
"""

print("--- Generating Edge Case Tests ---")
if app_code:
    generated_edge_case_tests = get_completion(edge_case_tests_prompt, client, model_name, api_provider)
    cleaned_edge_case_tests = clean_llm_output(generated_edge_case_tests, language='python')
    print(cleaned_edge_case_tests)
else:
    print("Skipping test generation because app code is missing.")